## Synthesizing output from matches

In [1]:
from sppysound.database import AudioDatabase, Synthesizer
import synthesis_config

In [2]:
source_dir = "./ExampleDatabase"
target_dir = "./ExampleTarget"
output_dir = "./ExampleOutput"

Load source database.
Also load the F0, RMS and Peak analyses for use with amplitude and pitch enforcement.

In [3]:
source_database = AudioDatabase(
    source_dir,
    config=synthesis_config,
    analysis_list={"f0", "rms", "peak"}
)
source_database.load_database(reanalyse=False)

Load database used to generate matches to source database. 
This is used when enforcing analyses such as RMS and F0. (Original grains are needed to calculate the ratio to alter the synthesized grain by)

In [4]:
target_database = AudioDatabase(
    target_dir,
    config=synthesis_config,
    analysis_list={"f0", "rms", "peak"}
)
target_database.load_database(reanalyse=False)

output_database = AudioDatabase(
    output_dir,
    config=synthesis_config
)
output_database.load_database(reanalyse=False)

Initialise the synthesizer object used for generating the final output.

In [5]:
synthesizer = Synthesizer(source_database, output_database, target_db=target_database, config=synthesis_config)

Run synthesis. As with the matching, warnings may be generated. These have all been accounted for and will be silenced in a future release. The output audio can now be found in the audio folder of ./ExampleOutput

In [6]:
synthesizer.synthesize()

/Users/sam/PerryPerrySource/pysource/pysound/src/sppysound/audiofile.py:665: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.times == None:
/Users/sam/PerryPerrySource/pysource/pysound/src/sppysound/audiofile.py:297: UserWarning: write_frames::warning::audio data has been clipped while writing to file ./.shift_input.wav.
  return self.pysndfile_object.write_frames(input)
/Users/sam/.pyenv/versions/2.7.10/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/sam/.pyenv/versions/2.7.10/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The synthesis_config.py file for this demo is:

~~~python
rms = {
    "window_size": 100,
    "overlap": 2,
}

analysis_dict = {
    "f0": "log2_median",
    "rms": "mean"
}

analysis = {
    "reanalyse": False
}

output_file = {
    "samplerate": 44100,
    "format": 131075,
    "channels": 1
}

synthesizer = {
    "enforce_rms": True,
    "enf_rms_ratio_limit": 5.,
    "enforce_f0": True,
    "enf_f0_ratio_limit": 10.,
    "grain_size": 100,
    "overlap": 2,
    "normalize" : True,
    # Defines the number of potential grains to choose from matches when
    # synthesizing output.
    "match_quantity": 20
}
~~~